<a href="https://colab.research.google.com/github/Wukkkinz-0725/animalImage_classification/blob/master/EfficientNetB0_KNN_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Wukkkinz-0725/animalImage_classification.git

Cloning into 'animalImage_classification'...
remote: Enumerating objects: 18600, done.
remote: Counting objects: 100% (18600/18600), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 18600 (delta 18556), reused 18582 (delta 18549), pack-reused 0
Receiving objects: 100% (18600/18600), 13.78 MiB | 20.73 MiB/s, done.
Resolving deltas: 100% (18556/18556), done.


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [ ]:
os.chdir('./animalImage_classification/Released_Data')

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image
import os
import pandas as pd

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])
cifar10_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Filter images evenly across remaining 7 classes
class_limit = 3000 // 7  # Approximate number of images per class
class_counts = {label: 0 for label in range(10) if label not in [2, 5, 6]}
filtered_images = []

for image, label in cifar10_dataset:
    if label in class_counts and class_counts[label] < class_limit:
        filtered_images.append((image, label))
        class_counts[label] += 1
    if all(count == class_limit for count in class_counts.values()):
        break

# Read existing data from CSV file or create a new DataFrame if the file doesn't exist
csv_file = 'train_data.csv'
if os.path.isfile(csv_file):
    existing_df = pd.read_csv(csv_file)
else:
    existing_df = pd.DataFrame(columns=['image', 'superclass_index', 'subclass_index'])

# Prepare CSV data for new images
new_csv_data = {'image': [], 'superclass_index': [], 'subclass_index': []}
if not os.path.exists('train_shuffle'):
    os.makedirs('train_shuffle')

for i, (image, _) in enumerate(filtered_images, start=6322):
    file_name = f'{i}.jpg'
    image_path = os.path.join('train_shuffle', file_name)
    image = transforms.ToPILImage()(image)
    image.save(image_path)

    # Update CSV data
    new_csv_data['image'].append(file_name)
    new_csv_data['superclass_index'].append(3)
    new_csv_data['subclass_index'].append(87)

# Create a DataFrame from the new data and append it to the existing DataFrame
new_df = pd.DataFrame(new_csv_data)
combined_df = existing_df.append(new_df, ignore_index=True)

# Write the combined DataFrame to the CSV file
combined_df.to_csv(csv_file, index=False)



100%|██████████| 170498071/170498071 [00:15<00:00, 11185596.99it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


<ipython-input-4-4f17ec5ab8b2>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = existing_df.append(new_df, ignore_index=True)


In [ ]:
import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

# 指定文件夹路径
folder_path = 'train_shuffle'

# 数文件数量
file_count = count_files(folder_path)
print(f"文件数量: {file_count}")


文件数量: 8322


In [ ]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [ ]:
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'


In [ ]:
image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

In [ ]:
# Augmentation and normalization for training
# Just normalization for validation and testing
train_transforms = transforms.Compose([
    transforms.RandomRotation(15),  # Rotates the image by +/- 15 degrees
    transforms.RandomHorizontalFlip(),  # Flips the image horizontally with 50% probability
    transforms.RandomVerticalFlip(),  # Flips the image vertically with 50% probability
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly changes image brightness, contrast, saturation, and hue
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization values can be adjusted
])

val_test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=train_transforms)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=val_test_transforms)

In [ ]:
# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [ ]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
                        nn.Conv2d(3, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(2, 2)
                      )

        self.block2 = nn.Sequential(
                        nn.Conv2d(32, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2, 2)
                      )

        self.block3 = nn.Sequential(
                        nn.Conv2d(64, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2, 2)
                      )

        self.fc1 = nn.Linear(4*4*128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3a = nn.Linear(128, 4)
        self.fc3b = nn.Linear(128, 88)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        super_out = self.fc3a(x)
        sub_out = self.fc3b(x)
        return super_out, sub_out



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CustomResNet18(nn.Module):
    def __init__(self):
        super(CustomResNet18, self).__init__()
        # Load the pre-trained ResNet18 model
        self.base_model = models.resnet18(pretrained=True)

        # ResNet18's last fc layer's in_features
        in_features = self.base_model.fc.in_features

        # Replace the original fully connected layer with new classifiers
        self.super_class_classifier = nn.Linear(in_features, 4)   # For super class
        self.sub_class_classifier = nn.Linear(in_features, 88)    # For sub class

        # Replace the original fc layer with a dummy layer that just passes through the features
        self.base_model.fc = nn.Identity()

    def forward(self, x):
        # Extract features from the base model
        features = self.base_model(x)

        # Classify into super and sub classes
        super_class_output = self.super_class_classifier(features)
        sub_class_output = self.sub_class_classifier(features)

        return super_class_output, sub_class_output


In [ ]:
class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda',superclass_threshold=0.9,subclass_threshold=0.3):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.superclass_threshold = superclass_threshold
        self.subclass_threshold = subclass_threshold

    def train_epoch(self):
        running_loss = 0.0
        device = self.device
        self.model.train()
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        device = self.device
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

                super_outputs, sub_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                super_probs = torch.nn.functional.softmax(super_outputs, dim=1)
                sub_probs = torch.nn.functional.softmax(sub_outputs, dim=1)
                # print(super_probs)
                # print(sub_probs)
                _, super_predicted = torch.max(super_probs, 1)
                _, sub_predicted = torch.max(sub_probs, 1)
                # Apply threshold to determine if a class is novel
                super_predicted[torch.max(super_probs, 1).values < self.superclass_threshold] = 3  # Novel superclass
                sub_predicted[torch.max(sub_probs, 1).values < self.subclass_threshold] = 87       # Novel subclass

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                super_outputs, sub_outputs = self.model(inputs)
                super_probs = torch.nn.functional.softmax(super_outputs, dim=1)
                sub_probs = torch.nn.functional.softmax(sub_outputs, dim=1)
                _, super_predicted = torch.max(super_probs, 1)
                _, sub_predicted = torch.max(sub_probs, 1)


                # Apply threshold to determine if a class is novel
                super_predicted[torch.max(super_probs, 1).values < self.superclass_threshold] = 3  # Novel superclass
                sub_predicted[torch.max(sub_probs, 1).values < self.subclass_threshold] = 87       # Novel subclass

                test_predictions['image'].append(img_name[0])
                test_predictions['superclass_index'].append(super_predicted.item())
                test_predictions['subclass_index'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)
        # Create separate DataFrames for superclass and subclass predictions
        superclass_predictions_df = test_predictions[['image', 'superclass_index']].rename(columns={'image': 'ID', 'superclass_index': 'Target'})
        subclass_predictions_df = test_predictions[['image', 'subclass_index']].rename(columns={'image': 'ID', 'subclass_index': 'Target'})
        if save_to_csv:
            superclass_predictions_df.to_csv('superclass_predict.csv', index=False)
            subclass_predictions_df.to_csv('subclass_predict.csv', index=False)

        if return_predictions:
            return superclass_predictions_df, subclass_predictions_df

In [ ]:
class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        device = self.device
        self.model.train()
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        device = self.device
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)

                super_outputs, sub_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        self.model.eval()
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(device), data[1]

                super_outputs, sub_outputs = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                test_predictions['image'].append(img_name[0])
                test_predictions['superclass_index'].append(super_predicted.item())
                test_predictions['subclass_index'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)
        # Create separate DataFrames for superclass and subclass predictions
        superclass_predictions_df = test_predictions[['image', 'superclass_index']].rename(columns={'image': 'ID', 'superclass_index': 'Target'})
        subclass_predictions_df = test_predictions[['image', 'subclass_index']].rename(columns={'image': 'ID', 'subclass_index': 'Target'})
        if save_to_csv:
            superclass_predictions_df.to_csv('superclass_predict.csv', index=False)
            subclass_predictions_df.to_csv('subclass_predict.csv', index=False)

        if return_predictions:
            return superclass_predictions_df, subclass_predictions_df

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CustomResNet34(nn.Module):
    def __init__(self):
        super(CustomResNet34, self).__init__()
        # Load the pre-trained ResNet34 model
        self.base_model = models.resnet34(pretrained=True)

        # ResNet34's last fc layer's in_features
        in_features = self.base_model.fc.in_features

        # Replace the original fully connected layer with new classifiers
        self.super_class_classifier = nn.Linear(in_features, 4)   # For super class
        self.sub_class_classifier = nn.Linear(in_features, 88)    # For sub class

        # Replace the original fc layer with a dummy layer that just passes through the features
        self.base_model.fc = nn.Identity()

    def forward(self, x):
        # Extract features from the base model
        features = self.base_model(x)

        # Classify into super and sub classes
        super_class_output = self.super_class_classifier(features)
        sub_class_output = self.sub_class_classifier(features)

        return super_class_output, sub_class_output


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CustomResNet50(nn.Module):
    def __init__(self):
        super(CustomResNet50, self).__init__()
        # Load the pre-trained ResNet50 model
        self.base_model = models.resnet50(pretrained=True)

        # ResNet50's last fc layer's in_features
        in_features = self.base_model.fc.in_features

        # Replace the original fully connected layer with new classifiers
        self.super_class_classifier = nn.Linear(in_features, 4)   # For super class
        self.sub_class_classifier = nn.Linear(in_features, 88)    # For sub class

        # Replace the original fc layer with a dummy layer that just passes through the features
        self.base_model.fc = nn.Identity()

    def forward(self, x):
        # Extract features from the base model
        features = self.base_model(x)

        # Classify into super and sub classes
        super_class_output = self.super_class_classifier(features)
        sub_class_output = self.sub_class_classifier(features)

        return super_class_output, sub_class_output


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CustomResNet152(nn.Module):
    def __init__(self):
        super(CustomResNet152, self).__init__()
        # Load the pre-trained ResNet152 model
        self.base_model = models.resnet152(pretrained=True)
        # Disable training for all layers
        for param in self.base_model.parameters():
            param.requires_grad = False
        # ResNet152's last fc layer's in_features
        in_features = self.base_model.fc.in_features

        # Replace the original fully connected layer with new classifiers
        self.super_class_classifier = nn.Linear(in_features, 4)   # For super class
        self.sub_class_classifier = nn.Linear(in_features, 88)    # For sub class

        # Replace the original fc layer with a dummy layer that just passes through the features
        self.base_model.fc = nn.Identity()

    def forward(self, x):
        # Extract features from the base model
        features = self.base_model(x)

        # Classify into super and sub classes
        super_class_output = self.super_class_classifier(features)
        sub_class_output = self.sub_class_classifier(features)

        return super_class_output, sub_class_output


In [ ]:
# Init model and trainer
device = 'cuda'
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [ ]:
device = 'cuda'
model = CustomResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 72.6MB/s]


In [ ]:
#Try resNet34
device = 'cuda'
model = CustomResNet34().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 94.2MB/s]


In [ ]:
#Try resNet50
model = CustomResNet50().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 98.6MB/s]


In [ ]:
#Try resNet152
device = 'cuda'
model = CustomResNet152().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 128MB/s]


In [ ]:
print(trainer.model)

CustomResNet18(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [ ]:
# Training loop
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1
Training loss: 3.492
Validation loss: 3.073
Validation superclass acc: 79.05 %
Validation subclass acc: 44.68 %

Epoch 2
Training loss: 2.640
Validation loss: 2.799
Validation superclass acc: 80.34 %
Validation subclass acc: 45.76 %

Epoch 3
Training loss: 2.265
Validation loss: 2.455
Validation superclass acc: 84.64 %
Validation subclass acc: 54.14 %

Epoch 4
Training loss: 2.105
Validation loss: 2.986
Validation superclass acc: 75.94 %
Validation subclass acc: 43.39 %

Epoch 5
Training loss: 2.001
Validation loss: 2.109
Validation superclass acc: 86.79 %
Validation subclass acc: 57.14 %

Epoch 6
Training loss: 1.835
Validation loss: 2.129
Validation superclass acc: 87.22 %
Validation subclass acc: 58.54 %

Epoch 7
Training loss: 1.845
Validation loss: 1.895
Validation superclass acc: 89.37 %
Validation subclass acc: 61.22 %

Epoch 8
Training loss: 1.772
Validation loss: 1.867
Validation superclass acc: 89.90 %
Validation subclass acc: 61.76 %

Epoch 9
Training loss: 1.648
Val

In [ ]:
trainer.test(save_to_csv=True, return_predictions=True)

'''
This simple baseline scores the following test accuracy

Superclass Accuracy
Overall: 43.83 %
Seen: 61.11 %
Unseen: 0.00 %

Subclass Accuracy
Overall: 2.03 %
Seen: 9.56 %
Unseen: 0.00 %
'''

'\nThis simple baseline scores the following test accuracy\n\nSuperclass Accuracy\nOverall: 43.83 %\nSeen: 61.11 %\nUnseen: 0.00 %\n\nSubclass Accuracy\nOverall: 2.03 %\nSeen: 9.56 %\nUnseen: 0.00 %\n'